In [ ]:
#include <iostream>
#include <vector>
#include <cmath>
#include <cstdlib>
#include <ctime>

struct Point {
    float x, y;
};

struct Circle {
    Point center;
    float radius;
    float probability;
};

// Function to perform Canny edge detection (simplified version)


In [ ]:
std::vector<Point> cannyEdgeDetection(const std::vector<std::vector<int>>& image) {
    std::vector<Point> edges;
    // Simplified edge detection: add all edge points to the vector
    for (int y = 1; y < image.size() - 1; ++y) {
        for (int x = 1; x < image[0].size() - 1; ++x) {
            if (image[y][x] == 1) {
                edges.push_back({x, y});
            }
        }
    }
    return edges;
}

// Function to calculate the circle parameters given three points


In [ ]:
Circle calculateCircle(const Point& p1, const Point& p2, const Point& p3) {
    float x1 = p1.x, y1 = p1.y;
    float x2 = p2.x, y2 = p2.y;
    float x3 = p3.x, y3 = p3.y;

    float a = x1 * (y2 - y3) - y1 * (x2 - x3) + (x2 * y3 - x3 * y2);
    if (std::abs(a) < 1e-6) {
        return {{0, 0}, 0, 0}; // Collinear points
    }

    float b = (x1 * x1 + y1 * y1) * (y3 - y2) + (x2 * x2 + y2 * y2) * (y1 - y3) + (x3 * x3 + y3 * y3) * (y2 - y1);
    float c = (x1 * x1 + y1 * y1) * (x2 - x3) + (x2 * x2 + y2 * y2) * (x3 - x1) + (x3 * x3 + y3 * y3) * (x1 - x2);
    float d = (x1 * x1 + y1 * y1) * (x3 * y2 - x2 * y3) + (x2 * x2 + y2 * y2) * (x1 * y3 - x3 * y1) + (x3 * x3 + y3 * y3) * (x2 * y1 - x1 * y2);

    float centerX = -b / (2 * a);
    float centerY = -c / (2 * a);
    float radius = std::sqrt((b * b + c * c - 4 * a * d) / (4 * a * a));

    return {{static_cast<float>(centerX), static_cast<float>(centerY)}, radius, 0};
}

// Learning Automata-based circle detection


In [ ]:
// Learning Automata-based circle detection
void detectCircles(const std::vector<Point>& edges, std::vector<Circle>& circles, int numCircles, int numIterations, float learningRate) {
    // Initialize circles with random triplets of edge points
    for (int i = 0; i < numCircles; ++i) {
        float idx1 = rand() % edges.size();
        float idx2 = rand() % edges.size();
        float idx3 = rand() % edges.size();
        cout << Form(" this is %f  and %f and %f ",edges[idx1].x, edges[idx2].x,edges[idx3].x) << endl;

        circles.push_back(calculateCircle(edges[idx1], edges[idx2], edges[idx3]));
    }

    // Learning automata iterations
    for (int iter = 0; iter < numIterations; ++iter) {
        for (auto& circle : circles) {
            // Evaluate circle (simplified: number of edge points close to the circle)
            int count = 0;
            for (const auto& p : edges) {
                float distance = std::sqrt(std::pow(p.x - circle.center.x, 2) + std::pow(p.y - circle.center.y, 2));
                if (std::abs(distance - circle.radius) < 2) { // Tolerance of 2 pixels
                    count++;
                }
            }

            // Update probability based on the count of close points
            circle.probability += learningRate * count;
        }

        // Normalize probabilities
        float sumProb = 0;
        for (const auto& circle : circles) {
            sumProb += circle.probability;
        }
        for (auto& circle : circles) {
            circle.probability /= sumProb;
        }
    }
}

In [ ]:

TFile * File  = TFile::Open("../TB_23/20231017-190808/recodata.root");


//auto myfile = 
TTree * Tree= (TTree*) File->Get("recodata");


    
 //init branches
float x[60000];
float y[60000];
float t[60000];
UShort_t n;
    // set branch address
cout << Tree->GetEntries() << endl;
   // TGraphErrors* gr = new TGraphErrors();
Tree->SetBranchAddress("x",&x);
Tree->SetBranchAddress("y",&y);
Tree->SetBranchAddress("t",&t);
Tree->SetBranchAddress("n",&n);
//tree entry loop


TH2D * xyhist = new TH2D("xyhist",";X position; Y position", 60,-100,100,60,-100,100); 
std::vector<Point> points;
for(int i=0; i<Tree->GetEntries();i++){
    
   Tree->GetEntry(i);
  
   if(i%1000==0)  cout << " x "<< x[0] << " y " << y[0] << " n "<< n<< "  t " << t[0]<< " "<< i <<endl;
    
   for(int j = 0; j< n ; j++){
         if((t[j])<-10) continue;
       
         if((t[j])>30) continue;
       
        
         xyhist->Fill(x[j], y[j]);
         points.push_back({x[j], y[j]});
         
         
      
    } 
    
    
}


In [ ]:
 // Parameters for the Learning Automata
    int numCircles = 10; // Number of circle candidates
    int numIterations = 100; // Number of iterations for the learning automata
    float learningRate = 0.01f;

    // Step 2: Circle detection using Learning Automata
    std::vector<Circle> circles;
    detectCircles(points, circles, numCircles, numIterations, learningRate);

In [ ]:
cout<< circles.size()<< endl;

In [ ]:
for (const auto& circle : circles) {
        std::cout << "Circle: Center=(" << circle.center.x << "," << circle.center.y << "), Radius=" << circle.radius << ", Probability=" << circle.probability << std::endl;
    }

In [ ]:
// Function to select the top N circles with the highest probabilities
std::vector<Circle> selectTopCircles(const std::vector<Circle>& circles, int topN) {
    std::vector<Circle> sortedCircles = circles;
    // Sort circles by probability in descending order
    std::sort(sortedCircles.begin(), sortedCircles.end(), [](const Circle& a, const Circle& b) {
        return a.probability > b.probability;
    });

    // Select the top N circles
    if (topN > sortedCircles.size()) {
        topN = sortedCircles.size();
    }
    return std::vector<Circle>(sortedCircles.begin(), sortedCircles.begin() + topN);
}

In [ ]:
auto mycircle = selectTopCircles(circles,50);

In [ ]:
TArc * arc = new TArc(mycircle[0].center.x,mycircle[0].center.x,mycircle[0].radius);
arc->SetFillStyle(0);

TArc * arc2 = new TArc(mycircle[1].center.x,mycircle[1].center.x,mycircle[1].radius);
arc2->SetFillStyle(0);

TArc * arc3 = new TArc(mycircle[11].center.x,mycircle[11].center.x,mycircle[11].radius);
arc3->SetFillStyle(0);


TCanvas * can = new TCanvas("can","",800,800);
can->cd();
xyhist->Draw();
arc->Draw("SAME");
arc2->Draw("SAME");
arc3->Draw("SAME");

can->Draw();